In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
# from model_utils.util_global_struct import process_bb_old_to_new

In [ ]:
from model_utils.utils_model_s1_cvae_global_z_bp_matrix import Predictor

In [ ]:


model_path = 's1_training/result/run_7/model_ckpt_ep_99.pth'


predictor = Predictor(model_path, latent_dim=50)


In [ ]:
df = pd.read_pickle('data/synthetic_bb_dist.len50.num100.sample10.pkl.gz')

In [ ]:

df = df[df['seq_id'] == 1]

assert df['seq'].nunique() == 1
seq = df.iloc[0]['seq']
one_idxs = df['one_idx'].tolist()
print(len(seq))

In [ ]:
df

In [ ]:
y_pred, mu_p, logvar_p, z = predictor.predict_matrix(seq)
px.imshow(y_pred)

In [ ]:
print(z)

In [ ]:
print(mu_p)

In [ ]:
print(logvar_p)

In [ ]:
# ground truth bp matrix

# lazy way to get all unique bp matrices
y_target_all = []
for one_idx in one_idxs:
    tmp = np.zeros((len(seq), len(seq)))
    tmp[one_idx] = 1
    if any([np.array_equal(a, tmp) for a in y_target_all]):
        continue
    y_target_all.append(tmp)

In [ ]:
print(len(y_target_all))

In [ ]:



fig = make_subplots(rows=2, cols=5)

for i, y_target in enumerate(y_target_all):
    idx_row = i // 5
    idx_col = i % 5
    fig.add_trace(
        px.imshow(y_target_all[i][::-1,:]).data[0],  # hacky, flip vertically so subplot is un-flipped
        row=idx_row+1, col=idx_col+1
    )

# fig.update_layout(yaxis=dict(autorange="reversed"))
# fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()

In [ ]:
for i, y_target in enumerate(y_target_all):
    print(roc_auc_score(y_score=y_pred.flatten(), y_true=y_target.flatten()))

In [ ]:
# px.histogram(y_pred[y_target_all[0]==1])
sorted(y_pred[y_target_all[0]==1], reverse=True)

In [ ]:
# px.histogram(y_pred[y_target_all[0]==0])
sorted(y_pred[y_target_all[0]==0], reverse=True)